In [134]:
import pandas as pd
import numpy as np
import pycountry
import translators as ts
import unicodedata
from tqdm.notebook import tqdm
tqdm.pandas()

def get_iso_code(country):
    try:
        country_name = pycountry.countries.search_fuzzy(country["country"].values[0])[0]
        return country_name.alpha_3
    except LookupError:
        return codes[country["País"].values[0]]
 
    
def clean_country_name(country_name):
    country_name = country_name[0]
    country_name = country_name.encode('latin-1').decode('utf-8')
    return country_name

def translate_country(country_name):
    new_name = ts.translate_text(country_name)
    print(new_name)
    return new_name
    
new_country_names = {
	"Qatar" : "CATAR",
	"República Checa" : "CHEQUIA",
	"Antilhas Holandesas" : "CURAÇAU",
	"Egipto":"EGITO",
	"EUA" : "ESTADOS UNIDOS DA AMERICA",
	"Rússia" : "FEDERACAO DA RUSSIA",
	"Reino Unido" : "REINO UNIDO DA GRA-BRETANHA E IRLANDA DO NORTE",
    "China (não inclui Macau e Hong Kong)" : "China",
    "China – Hong Kong" : "Hong Kong",
    "China – Macau":"Macau",
    "china - hong kong" : "Hong King",
    "china - macau" : "Macau"
	
}

codes = {
	'AFRICA ORIENTAL': np.nan,
	'CABO VERDE': "CPV",
	'MACAU': "MAC",
    'CONGO (REPUBLICA DEMOCRATICA DO)': "COD",
    'COSTA DO MARFIM': "CIV",
    'GANA': "GHA",
    'GRANADA': "GRD",
    'ILHAS MARSHALL': "MHL",
    'LAOS': "LAO",
    'PALAU': "PLW",
    'PALESTINIAN TERRITORY, OCCUPIED': "PSE",
    'REUNION REUNION': "REU",
    'SAO BARTOLOMEU': "BLM",
    'SUAZILANDIA': "SWZ",
    'TIMOR-LESTE': "TLS"

}
    
entradas = pd.read_excel('dados/entradas_portugueses.xlsx').rename({"Unnamed: 0": "País"}, axis = 1)
consulados = pd.read_csv("consulados.csv")

In [18]:
consulados = pd.read_csv("consulados.csv")
consulados["CONSULADO"] = consulados["CONSULADO"].astype(str).str[:4]
consulados = consulados[["CONSULADO", "Pais"]]
consulados = consulados.rename({"CONSULADO" : "codigo", "Pais": "pais"}, axis = 1)
consulados = consulados.set_index("codigo")
consulados["pais"] = consulados["pais"].str.rstrip()
consulados["pais"] = consulados["pais"].str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8').str.upper()
consulados['pais'] = consulados['pais'].str.split(' \(', expand=True)[0]
correspondencias = dict(zip(consulados["pais"],consulados.index))

entradas["País"] = entradas["País"].str.rstrip()
entradas["País"] = entradas["País"].replace(new_country_names)

In [ ]:
entradas['country'] = entradas['País'].apply(lambda x: ts.translate_text(x))
entradas["País"] = entradas["País"].str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8').str.upper()
entradas["MAI_code"] = entradas["País"].map(correspondencias)
entradas['ISO_code'] = entradas['country'].apply(get_iso_code)
df = entradas

In [ ]:


entradas = entradas.drop(columns="country")

entradas = entradas.melt(id_vars=['País', "ISO_code", "MAI_code"], var_name='Ano', value_name="Entradas")
entradas = entradas.sort_values(by=["País", "Ano"], ascending = [True, False])

entradas = entradas.set_index(["País", "Ano"])

In [ ]:
entradas.to_csv("entradas.csv")

In [135]:
residentes = pd.read_html("dados/residentes_idade.html")

residentes = residentes[0]
residentes = residentes.replace("..", np.nan)

a = residentes.columns.get_level_values(0)
b = residentes.columns.get_level_values(1)

residentes.columns = [a.to_series().mask(lambda x: x.str.startswith('Unnamed')).ffill(), b]
residentes = residentes.rename(columns={"Unnamed: 0_level_1":"País"})

residentes = residentes.set_index((np.nan, "País"))
residentes.index.names = ["País"]
residentes = residentes.reset_index()

residentes["País"] = residentes["País"].replace(new_country_names)
residentes['country'] = residentes['País'].apply(lambda x: translate_country(x))
df = residentes

#residentes["País"].values

Afghanistan
South Africa
East Africa
Albania
Germany
American Samoa
Andorra
Angola
Anguilla
Antarctica
Antigua and Barbuda
CURAÇAO
Saudi Arabia
Algeria
Argentina
Armenia
Aruba
Australia
Austria
Azerbaijan
Bahamas
Bahrain
Bangladesh
Barbados
Belgium
Belize
Benin
Bermuda
Belarus
Bolivia
Bosnia and Herzegovina
Botswana
Bouvet Island
Brazil
British Indian Ocean Territory
Brunei
Bulgaria
Burkina Faso
Burundi
Bhutan
Cape Verde
Cameroon
Cambodia
Canada
Kazakhstan
Chad
Chile
China
Hong Kong
Macau
Cyprus
Christmas Island
Cocos (Keeling) Islands
Colombia
Comoros
Congo
Congo (Democratic Republic of)
Cook Islands
Korea
North Korea
Ivory Coast
Costa Rica
Croatia
Cuba
Denmark
Djibouti
Dominica
EGYPT
El Salvador
United Arab Emirates
Ecuador
Eritrea
Slovakia
Slovenia
Spain
Estonia
Ethiopia
UNITED STATES OF AMERICA
Falkland Islands (Malvinas)
Faroe Islands
Fiji
Philippines
Finland
France
French Southern Territories
Gabon
Gambia
Desire
Georgia
Gibraltar
Grenade
Greece
Greenland
Guadeloupe
Guam
Guatemala

In [136]:
residentes["País"] = residentes["País"].str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8').str.upper()
residentes["MAI_code"] = residentes["País"].map(correspondencias)
residentes['ISO_code'] = residentes.apply(get_iso_code, axis = 1)
residentes = residentes.drop(columns="country")

residentes

/var/folders/2_/dc69y37n7d14q5rbg8ly14g40000gn/T/ipykernel_66916/2893637283.py:4: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  residentes = residentes.drop(columns="country")


País  2023                                 2022          
                          Total < 15 15-39 40-64 15-64 > 65    Total   < 15   
0             AFEGANISTAO   NaN  NaN   NaN   NaN   NaN  NaN      NaN    NaN  \
1           AFRICA DO SUL   NaN  NaN   NaN   NaN   NaN  NaN      NaN    NaN   
2         AFRICA ORIENTAL   NaN  NaN   NaN   NaN   NaN  NaN      NaN    NaN   
3                 ALBANIA   NaN  NaN   NaN   NaN   NaN  NaN      NaN    NaN   
4                ALEMANHA   NaN  NaN   NaN   NaN   NaN  NaN  139 425  9 220   
..                    ...   ...  ...   ...   ...   ...  ...      ...    ...   
242  VIRGIN ISLANDS, U.S.   NaN  NaN   NaN   NaN   NaN  NaN      NaN    NaN   
243     WALLIS AND FUTUNA   NaN  NaN   NaN   NaN   NaN  NaN      NaN    NaN   
244        WESTERN SAHARA   NaN  NaN   NaN   NaN   NaN  NaN      NaN    NaN   
245                ZAMBIA   NaN  NaN   NaN   NaN   NaN  NaN      NaN    NaN   
246              ZIMBABUE   NaN  NaN   NaN   NaN   NaN  NaN      NaN    NaN   

                                      2021                                
      15-39   40-64 15-64    > 65    Total   < 15   15-39   40-64 15-64   
0       NaN     NaN   NaN     NaN      NaN    NaN     NaN     NaN   NaN  \
1       NaN     NaN   NaN     NaN      NaN    NaN     NaN     NaN   NaN   
2       NaN     NaN   NaN     NaN      NaN    NaN     NaN     NaN   NaN   
3       NaN     NaN   NaN     NaN      NaN    NaN     NaN     NaN   NaN   
4    45 370  65 355   NaN  19 480  138 740  9 320  45 670  64 775   NaN   
..      ...     ...   ...     ...      ...    ...     ...     ...   ...   
242     NaN     NaN   NaN     NaN      NaN    NaN     NaN     NaN   NaN   
243     NaN     NaN   NaN     NaN      NaN    NaN     NaN     NaN   NaN   
244     NaN     NaN   NaN     NaN      NaN    NaN     NaN     NaN   NaN   
245     NaN     NaN   NaN     NaN      NaN    NaN     NaN     NaN   NaN   
246     NaN     NaN   NaN     NaN      NaN    NaN     NaN     NaN   NaN   

                2020                                          2019          
       > 65    Total   < 15   15-39   40-64 15-64    > 65    Total   < 15   
0       NaN      NaN    NaN     NaN     NaN   NaN     NaN      NaN    NaN  \
1       NaN      NaN    NaN     NaN     NaN   NaN     NaN      NaN    NaN   
2       NaN      NaN    NaN     NaN     NaN   NaN     NaN      NaN    NaN   
3       NaN      NaN    NaN     NaN     NaN   NaN     NaN      NaN    NaN   
4    18 975  138 560  9 430  46 405  64 115   NaN  18 610  138 400  9 440   
..      ...      ...    ...     ...     ...   ...     ...      ...    ...   
242     NaN      NaN    NaN     NaN     NaN   NaN     NaN      NaN    NaN   
243     NaN      NaN    NaN     NaN     NaN   NaN     NaN      NaN    NaN   
244     NaN      NaN    NaN     NaN     NaN   NaN     NaN      NaN    NaN   
245     NaN      NaN    NaN     NaN     NaN   NaN     NaN      NaN    NaN   
246     NaN      NaN    NaN     NaN     NaN   NaN     NaN      NaN    NaN   

                                      2018                                
      15-39   40-64 15-64    > 65    Total   < 15   15-39   40-64 15-64   
0       NaN     NaN   NaN     NaN      NaN    NaN     NaN     NaN   NaN  \
1       NaN     NaN   NaN     NaN      NaN    NaN     NaN     NaN   NaN   
2       NaN     NaN   NaN     NaN      NaN    NaN     NaN     NaN   NaN   
3       NaN     NaN   NaN     NaN      NaN    NaN     NaN     NaN   NaN   
4    47 490  63 315   NaN  18 155  138 890  9 560  49 035  62 575   NaN   
..      ...     ...   ...     ...      ...    ...     ...     ...   ...   
242     NaN     NaN   NaN     NaN      NaN    NaN     NaN     NaN   NaN   
243     NaN     NaN   NaN     NaN      NaN    NaN     NaN     NaN   NaN   
244     NaN     NaN   NaN     NaN      NaN    NaN     NaN     NaN   NaN   
245     NaN     NaN   NaN     NaN      NaN    NaN     NaN     NaN   NaN   
246     NaN     NaN   NaN     NaN      NaN    NaN     NaN     NaN   NaN   

                2017                   

In [137]:
melted_df = residentes.melt(id_vars = ["País", "ISO_code", "MAI_code"], value_name='Residentes', var_name=['Ano', 'Faixa Etária'])
melted_df = melted_df.set_index(["ISO_code", "Ano"])


In [138]:
melted_df = melted_df.dropna(subset=['Residentes'])
melted_df

df = melted_df

In [139]:
df = df.reset_index()
max_years = df.groupby('ISO_code')['Ano'].max().reset_index()

# Merge with the original DataFrame to keep only the rows with the most recent year for each country
df = pd.merge(df, max_years, on=['ISO_code', 'Ano'])

df = df.reset_index(drop=True)
df["Residentes"] = df["Residentes"].str.normalize('NFKD').str.replace(" ", "")
df["Residentes"] = df["Residentes"].astype(int)

,ISO_code,Ano,País,MAI_code,Faixa Etária,Residentes
0,DNK,2023,DINAMARCA,8107,Total,3 789
1,DNK,2023,DINAMARCA,8107,< 15,348
2,DNK,2023,DINAMARCA,8107,15-39,2 149
3,DNK,2023,DINAMARCA,8107,40-64,1 135
4,DNK,2023,DINAMARCA,8107,> 65,157
...,...,...,...,...,...,...
118,IRL,2016,IRLANDA,8114,Total,4 807
119,IRL,2016,IRLANDA,8114,< 15,559
120,IRL,2016,IRLANDA,8114,15-39,2 809
121,IRL,2016,IRLANDA,8114,40-64,1 385


In [179]:
df_teste = df
#df_teste["Residentes"] = df_teste["Residentes"].str.normalize('NFKD').str.replace(" ", "")
#df_teste["Residentes"] = df_teste["Residentes"].astype(int)
grouped = df.groupby(["ISO_code", "Ano"])

# Calculate the percentage of residents for each age group within each group
df["Residentes_Pct"] = grouped.apply(lambda x: (x["Residentes"] / x[x["Faixa Etária"] == "Total"]["Residentes"].values[0]) * 100)

TypeError: incompatible index of inserted column with frame index

In [183]:
df_teste["total_residents"] = df_teste.loc[df_teste["Faixa Etária"] == "Total", "Residentes"]

In [185]:
df_teste["total_residents"] = df_teste["total_residents"].ffill()

In [187]:
df_teste["proporcao_residentes"] = df_teste["Residentes"] / df_teste["total_residents"] * 100

In [189]:
df_teste["proporcao_residentes"] = df_teste["proporcao_residentes"].round(2)

In [190]:
df_teste = df_teste.drop(columns=["total_residents"])

df_teste

,ISO_code,Ano,País,MAI_code,Faixa Etária,Residentes,proporcao_residentes
0,DNK,2023,DINAMARCA,8107,Total,3789,100.00
1,DNK,2023,DINAMARCA,8107,< 15,348,9.18
2,DNK,2023,DINAMARCA,8107,15-39,2149,56.72
3,DNK,2023,DINAMARCA,8107,40-64,1135,29.96
4,DNK,2023,DINAMARCA,8107,> 65,157,4.14
...,...,...,...,...,...,...,...
118,IRL,2016,IRLANDA,8114,Total,4807,100.00
119,IRL,2016,IRLANDA,8114,< 15,559,11.63
120,IRL,2016,IRLANDA,8114,15-39,2809,58.44
121,IRL,2016,IRLANDA,8114,40-64,1385,28.81


In [191]:
df_teste = df_teste.set_index("ISO_code").to_csv("residentes_idades.csv")